In [ ]:
%cd ../..

# Get hists pipeline

In [ ]:
saved = "ml_hep_sim/analysis/results/pulls/"

In [ ]:
from ml_hep_sim.analysis.hists_pipeline import get_hists_pipeline
from ml_hep_sim.analysis.hists_pipeline import MakeHistsFromSamples

In [ ]:
use_class = True
mc_test = False

if use_class and not mc_test:
    saved += "class_"
elif use_class and mc_test:
    saved += "class_mc_"
elif mc_test:
    saved += "mbb_mc_"
else:
    saved += "mbb_"

hists_pipeline = get_hists_pipeline(use_classifier=use_class, N_gen=10**6)
hists_pipeline.pipes = hists_pipeline.pipes[:-1] # this one only makes 1 hist

In [ ]:
saved

In [ ]:
b_sig_bkg = hists_pipeline.pipes[-1]

# Do a scan for different N values

In [ ]:
import numpy as np

from ml_hep_sim.analysis.ul_pipeline import PullBlock
from ml_hep_sim.pipeline.pipes import Pipeline

from ml_hep_sim.analysis.ul_pipeline import pull_plot

In [ ]:
Ns = np.linspace(10**3, 10**5, 40).astype(int)

In [ ]:
pull_pipelines = []

bonly = False
bins = 22

sys_err = 0.1

for N in Ns:
    if use_class:
        bin_range = (0.5, 1.1)
        b_hists = MakeHistsFromSamples(bin_range=bin_range, bins=bins, N_sig=N / 10, N_bkg=N, 
                                       N_gen=10**6, bonly=bonly)(b_sig_bkg)
    else:
        bin_range = (0.01, 3.0)
        b_hists = MakeHistsFromSamples(bin_range=bin_range, bins=bins, N_sig=N / 10, N_bkg=N, 
                                       N_gen=10**6, bonly=bonly)(b_sig_bkg)
    
    b_pull = PullBlock(bkg_err=sys_err, mc_test=mc_test)(b_hists)
    pipe = Pipeline()
    pipe.compose(hists_pipeline, b_hists, b_pull)
    
    pull_pipelines.append(pipe)

In [ ]:
for pull_pipeline in pull_pipelines:
    pull_pipeline.fit()

In [ ]:
pull_pipelines[-1].draw_pipeline_tree(to_graphviz_file="ml_hep_sim/analysis/results/pulls/pull_pipe", block_idx=-1)

In [ ]:
pulls_lst, pullerr_lst, mus, mus_err, twice_nlls = [], [], [], [], []

for i, N in enumerate(Ns):
    print(N)
    p = pull_pipelines[i].pipes[-1]
    pulls, pullerr, errors, labels = p.results
    
    pull_plot(pulls, pullerr, errors, labels) #, save=saved + f"pull_{N}.pdf")
    # pull_plot(pulls, pullerr, errors, labels)
    
    pulls_lst.append(pulls)
    pullerr_lst.append(pullerr)
    mus.append(p.bestfit[0][0])
    mus_err.append(errors[0])
    twice_nlls.append(p.bestfit[1])

# Mu

In [ ]:
from uncertainties import unumpy, ufloat

import matplotlib.pyplot as plt
from ml_hep_sim.plotting.style import style_setup, set_size

set_size()
style_setup(seaborn_pallete=True)

In [ ]:
eff_s_mc, eff_b_mc, eff_s_ml, eff_b_ml = pull_pipelines[0].pipes[-2].histograms["eff"]

In [ ]:
plt.errorbar(Ns, mus, mus_err, capsize=5)
plt.scatter(Ns, mus)
plt.xlabel(r"$\nu_B$", loc="center")
plt.ylabel(r"$\mu$")
plt.tight_layout()
plt.axhline(0.1, c='r', ls='--')
# plt.savefig(saved + "mus.pdf")

In [ ]:
unumpy.uarray(mus, mus_err)

# Gamma means and stds

In [ ]:
gammas = [] # list of gamma and error for each N

for g_val, g_err in zip(pulls_lst, pullerr_lst):
    lst = []
    for v, err in zip(g_val, g_err):
        lst.append(ufloat(v, err))
    
    gammas.append(np.array(lst))

In [ ]:
g_res = []

for g in gammas:
    g_res.append(np.sum(g**2)**0.5)

In [ ]:
g_nom, g_err = [], [] # split to nominal and std

for g in g_res:
    g_nom.append(g.nominal_value)
    g_err.append(g.std_dev)

In [ ]:
plt.scatter(Ns, g_nom)
plt.errorbar(Ns, g_nom, g_err, capsize=5)
plt.xlabel(r"$\nu_B$", loc='center')
plt.ylabel(r"$\gamma$")
plt.tight_layout()
# plt.savefig(saved + "gamma_N.pdf")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs = axs.flatten()

axs[0].plot(Ns, g_nom, lw=3)
axs[1].plot(Ns, g_err, lw=3)

axs[0].set_ylabel(r"$\sum_b \gamma_b^2$")
axs[1].set_ylabel(r"$\sum_b \gamma_b^2$ errors")

axs[0].set_xlabel(r"$\nu_B$", loc='center')
axs[1].set_xlabel(r"$\nu_B$", loc='center')
plt.tight_layout()
# plt.savefig(saved + "gamma_N_err_class.pdf")

# Hists

In [ ]:
if mc_test is True:
    raise ValueError

In [ ]:
idx = 0
N_idx = Ns[idx]

pull_pipe = pull_pipelines[idx].pipes[-1] # idx depends on N

hists = pull_pipe.histograms
mu = pull_pipe.bestfit[0][0]
gamma = pull_pipe.bestfit[0][1:]
mu_err, gamma_err = pull_pipe.results[-2][0], pull_pipe.results[-2][1:]

hist_pipe = pull_pipelines[idx].pipes[-2]
bkg_gen = hist_pipe.bkg_generated_data

In [ ]:
alpha = hist_pipe.alpha

# Errors

In [ ]:
from ml_hep_sim.plotting.hep_plots import StackPlot

In [ ]:
errors = pull_pipe.errors

data_err = errors["data_mc"]
bkg_err = np.sqrt(errors["nu_b_ml"] ** 2 + (hists["bkg_gen"] * sys_err)**2)

# Prefit

In [ ]:
x = np.arange(0, bins, 1)

sp = StackPlot(
    x,
    hists_lst=[alpha * hists["sig_mc"], hists["bkg_gen"], alpha * hists["sig_mc"] + hists["bkg_gen"]],
    data_hist=alpha * hists["sig_mc"] + hists["bkg_gen"] + hists["data_mc"],
)

sp.plot_stack(labels=["MC sig", "ML bkg", "MC sig + ML bkg"])

sp.plot_data(label="MC data", err=data_err, fmt='.', capsize=1, lw=1)

sp.plot_mc_errors(bkg_err)

counts_num, counts_den = hists["data_mc"], alpha* hists["sig_mc"] + hists["bkg_gen"]

counts_num_err = data_err
counts_den_err = bkg_err

sp.plot_lower_panel(counts_num, counts_den, counts_num_err, counts_den_err, ylabel="data$/$ML",
                    label_x_start=bin_range[0],
                    label_x_end=bin_range[1])

ax = sp.ax
ax_lower = sp.ax_lower

if use_class:
    ax_lower.set_xlabel("class. output")
else:
    ax_lower.set_xlabel("$m_{bb}$")

ax.set_ylabel("$N$")

# ax.set_ylim(0, 250)
ax_lower.set_ylim(0.7, 1.3)

plt.legend(loc='upper right')
plt.tight_layout()

# plt.savefig(saved + f"stacked_prefit_{N_idx}.pdf")

# Postfit

In [ ]:
gammas = unumpy.uarray(gamma, gamma_err)
bkg = hists["bkg_gen"] # unumpy.uarray(hists["bkg_gen"], bkg_err)

bkg_postfit = bkg * gammas
sig_postfit = ufloat(mu, mu_err) * hists["sig_mc"]

bkg_postfit_err = np.sqrt(unumpy.std_devs(bkg_postfit) ** 2 + (bkg * sys_err) ** 2) # unumpy.std_devs(bkg_postfit)
sig_postfit_err = unumpy.std_devs(sig_postfit)

bkg_postfit_val = unumpy.nominal_values(bkg_postfit)
sig_postfit_val = unumpy.nominal_values(sig_postfit)

In [ ]:
sp = StackPlot(
    x,
    hists_lst=[sig_postfit_val, bkg_postfit_val, sig_postfit_val + bkg_postfit_val],
    data_hist=sig_postfit_val + bkg_postfit_val + hists["data_mc"],
)

sp.plot_stack(labels=["MC sig", "ML bkg", "MC sig + ML bkg"])

sp.plot_data(label="MC data", err=data_err, fmt='.', capsize=1, lw=1)

sp.plot_mc_errors(bkg_postfit_err)

counts_num, counts_den = hists["data_mc"], sig_postfit_val + bkg_postfit_val

counts_num_err = data_err
counts_den_err = bkg_postfit_err

sp.plot_lower_panel(counts_num, counts_den, counts_num_err, counts_den_err, ylabel="data$/$ML",
                    label_x_start=bin_range[0],
                    label_x_end=bin_range[1])

ax = sp.ax
ax_lower = sp.ax_lower

if use_class:
    ax_lower.set_xlabel("class. output")
else:
    ax_lower.set_xlabel("$m_{bb}$")

ax.set_ylabel("$N$")

# ax.set_ylim(0, 250)
ax_lower.set_ylim(0.7, 1.3)

plt.legend(loc='upper right')
plt.tight_layout()

# plt.savefig(saved + f"stacked_postfit_{N_idx}.pdf")

# Ratio

In [ ]:
plt.step(x, hists["bkg_gen"], where="mid", label="bkg gen")
plt.step(x, hists["sig_mc"], where="mid", label="sig mc") # scale by mu from pyhf
plt.step(x, hists["data_mc"], where="mid", label="data mc")
plt.step(x, hists["sig_mc"] + hists["bkg_gen"], where="mid", label="sig mc + bkg gen")
plt.legend()

In [ ]:
plt.step(x, hists["bkg_gen"], where="mid", label="bkg gen")
plt.step(x, mu * hists["sig_mc"], where="mid", label="sig mc") # scale by mu from pyhf
plt.step(x, hists["data_mc"], where="mid", label="data mc")
plt.step(x, mu * hists["sig_mc"] + hists["bkg_gen"], where="mid", label="sig mc + bkg gen")
plt.legend()

In [ ]:
plt.scatter(x, (alpha * hists["sig_mc"] + hists["bkg_gen"]) / hists["data_mc"], label="prefit")
plt.scatter(x, (mu * hists["sig_mc"] + gamma * hists["bkg_gen"]) / hists["data_mc"], label="postfit")
plt.xticks(x[::3], x[::3])
plt.ylabel("(MC sig + ML bkg) $/$ MC data")
plt.xlabel("bins")
plt.axhline(1, c='k')
plt.legend()
plt.tight_layout()
# plt.savefig(saved + f"ratio_pre_post_{N_idx}.pdf")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs = axs.flatten()

q = unumpy.uarray(mus, mus_err)

spur = q * Ns - alpha * Ns
y = unumpy.nominal_values(spur)
yerr = unumpy.std_devs(spur)

axs[0].scatter(Ns, y)
# axs[0].errorbar(Ns, y, yerr, capsize=5)

spur_ratio = (q * Ns - alpha * Ns) / Ns
y_ratio = unumpy.nominal_values(spur_ratio)
yerr_ratio = unumpy.std_devs(spur_ratio)

axs[1].scatter(Ns, y_ratio)
# axs[1].errorbar(Ns, y_ratio, yerr_ratio, capsize=5)

axs[0].set_ylabel(r"$S_\text{spur}=(\mu-\alpha)\nu_B$")
axs[1].set_ylabel(r"$\mu - \alpha = \frac{S_\text{spur}}{\nu_B}$")

axs[0].set_xlabel(r"$\nu_B$", loc='center')
axs[1].set_xlabel(r"$\nu_B$", loc='center')

plt.tight_layout()
# plt.savefig(saved + f"spur_expected.pdf")